Old file not to use


In [ ]:
import os
import cv2
import json
import numpy as np
import shutil
import torch
import pandas as pd
from tqdm import tqdm
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from scenedetect import detect, ContentDetector
import time

# Define paths
CLIPS_DIR = "/Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/Batman-(1989)-002"
FRAMES_DIR = "/Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/extracted-frames"
OUTPUT_DIR = "/Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/histogram-output-sequences"
RESULTS_CSV = "/Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/threshold_results.csv"

# Ensure required directories exist
os.makedirs(FRAMES_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load CLIP (SigLIP) model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Function to extract image embeddings using SigLIP
def get_image_embedding(image_path):
    """Extract embedding from an image using SigLIP (CLIP)."""
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        embedding = model.get_image_features(**inputs)
    return embedding.squeeze()

# Function to resize image while maintaining aspect ratio
def resize_image(image, target_size=246):
    """Resize image while keeping aspect ratio and fitting within target_size x target_size."""
    h, w = image.shape[:2]
    scale = target_size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    return cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)

# Step 1: Extract frames from each clip (with resizing)
def extract_frames_from_clip(clip_path, output_dir, frame_rate=1, target_size=246):
    """
    Extract frames from a video clip at a specified frame rate and resize them.

    Parameters:
        clip_path (str): Path to the video clip.
        output_dir (str): Directory to save extracted frames.
        frame_rate (int): Number of frames to extract per second.
        target_size (int): Resize frames to fit within target_size x target_size.

    Returns:
        int: Total frames extracted.
    """
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(clip_path)

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = max(1, fps // frame_rate)
    frame_count = 0
    extracted_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_resized = resize_image(frame, target_size)
            frame_path = os.path.join(output_dir, f"frame_{extracted_count:05d}.jpg")
            cv2.imwrite(frame_path, frame_resized)
            extracted_count += 1

        frame_count += 1

    cap.release()
    return extracted_count

# Step 2: Detect shot boundaries
def detect_shot_boundaries(video_path):
    print(f"\n🔍 Detecting shot boundaries in {os.path.basename(video_path)}...")
    scene_list = detect(video_path, ContentDetector())
    boundaries = [(scene[0].get_frames(), scene[1].get_frames()) for scene in scene_list]
    print(f"✅ Detected {len(boundaries)} sequences in {os.path.basename(video_path)}.\n")
    return boundaries

# Step 3: Organize frames into sequences
def organize_frames_by_sequence(clip_name, frame_dir, boundaries):
    print(f"📂 Organizing frames for {clip_name}...\n")
    frames = sorted([f for f in os.listdir(frame_dir) if f.endswith(".jpg") or f.endswith(".png")])
    
    sequences_metadata = {"clip": clip_name, "sequences": []}
    clip_output_dir = os.path.join(OUTPUT_DIR, clip_name)
    os.makedirs(clip_output_dir, exist_ok=True)

    for i, (start, end) in tqdm(enumerate(boundaries, 1), total=len(boundaries), desc="📂 Organizing Sequences"):
        seq_dir = os.path.join(clip_output_dir, f"sequence_{i}")
        os.makedirs(seq_dir, exist_ok=True)

        for frame in frames:
            frame_num = int(frame.split(".")[0].replace("frame_", ""))
            if start <= frame_num <= end:
                shutil.copy(os.path.join(frame_dir, frame), os.path.join(seq_dir, frame))

        sequences_metadata["sequences"].append({
            "sequence_id": i,
            "start_frame": start,
            "end_frame": end
        })

    # Save JSON file
    json_path = os.path.join(clip_output_dir, f"{clip_name}.json")
    with open(json_path, "w") as json_file:
        json.dump(sequences_metadata, json_file, indent=4)

    print(f"\n✅ JSON metadata saved at: {json_path}")


def extract_keyshots(sequence_dir, hist_threshold, siglip_threshold, method="BHATTACHARYYA"):
    """Extract keyshots using Histogram + SigLIP (CLIP) features efficiently."""
    frames = sorted([f for f in os.listdir(sequence_dir) if f.endswith((".jpg", ".png"))])
    keyshots = []
    
    if not frames:
        return keyshots

    prev_hist, prev_embedding = None, None

    for frame in tqdm(frames, desc=f"📸 Extracting Keyshots ({os.path.basename(sequence_dir)})", leave=False):
        frame_path = os.path.join(sequence_dir, frame)
        image = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)

        hist = cv2.calcHist([image], [0], None, [256], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()

        if prev_hist is None:
            keyshots.append(frame_path)
            prev_hist = hist
            prev_embedding = get_image_embedding(frame_path)
            continue

        hist_diff = cv2.compareHist(prev_hist, hist, cv2.HISTCMP_BHATTACHARYYA)

        if hist_diff > hist_threshold:
            if prev_embedding is None:
                prev_embedding = get_image_embedding(frame_path)
            
            curr_embedding = get_image_embedding(frame_path)
            similarity = torch.nn.functional.cosine_similarity(prev_embedding, curr_embedding, dim=0)

            if similarity < siglip_threshold:
                keyshots.append(frame_path)
                prev_embedding = curr_embedding

        prev_hist = hist

    return keyshots


# 🚀 **Main Execution**
if __name__ == "__main__":
    hist_thresholds = [0.2, 0.3, 0.4, 0.5]
    siglip_thresholds = [0.75, 0.8, 0.85, 0.9]
    results = []

    print("\n🔑 Processing multiple clips...\n")

    for clip_name in tqdm(os.listdir(CLIPS_DIR), desc="🎬 Processing Clips"):
        clip_path = os.path.join(CLIPS_DIR, clip_name)
        frame_dir = os.path.join(FRAMES_DIR, clip_name)

        if not os.path.isfile(clip_path):
            print(f"⚠️ Skipping {clip_name} (Not a file)")
            continue

        print(f"\n🎥 Extracting frames from {clip_name}...")
        total_frames = extract_frames_from_clip(clip_path, frame_dir)

        if total_frames == 0:
            print(f"⚠️ No frames extracted from {clip_name}. Skipping...")
            continue

        boundaries = detect_shot_boundaries(clip_path)
        organize_frames_by_sequence(clip_name, frame_dir, boundaries)

        for hist_thresh in hist_thresholds:
            for siglip_thresh in siglip_thresholds:
                start_time = time.time()
                clip_output_dir = os.path.join(OUTPUT_DIR, clip_name)

                # for seq in tqdm(os.listdir(clip_output_dir), desc=f"🔑 Processing Sequences ({clip_name})"):
                #     seq_path = os.path.join(clip_output_dir, seq)
                #     extract_keyshots(seq_path, hist_threshold=hist_thresh, siglip_threshold=siglip_thresh)
                if not os.path.isdir(clip_output_dir):
                    print(f"⚠️ Warning: {clip_output_dir} is not a directory. Skipping...")
                    continue

                for seq in tqdm(os.listdir(clip_output_dir), desc=f"🔑 Processing Sequences ({clip_name})"):
                    seq_path = os.path.join(clip_output_dir, seq)
                    if not os.path.isdir(seq_path):  # Ensure seq_path is a directory
                        continue
                    extract_keyshots(seq_path, hist_threshold=hist_thresh, siglip_threshold=siglip_thresh)

                runtime = round(time.time() - start_time, 2)
                results.append({"Clip": clip_name, "Hist_Threshold": hist_thresh, "Runtime": runtime})

    pd.DataFrame(results).to_csv(RESULTS_CSV, index=False)
    print("\n✅ Process complete!")


🔑 Processing multiple clips...



🎬 Processing Clips:   0%|          | 0/1073 [00:00<?, ?it/s]


🎥 Extracting frames from Batman (1989)-002_part_374.mp4...

🔍 Detecting shot boundaries in Batman (1989)-002_part_374.mp4...
✅ Detected 0 sequences in Batman (1989)-002_part_374.mp4.

📂 Organizing frames for Batman (1989)-002_part_374.mp4...



📂 Organizing Sequences: 0it [00:00, ?it/s]



✅ JSON metadata saved at: /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/histogram-output-sequences/Batman (1989)-002_part_374.mp4/Batman (1989)-002_part_374.mp4.json


...


🎥 Extracting frames from Batman (1989)-002_part_379.mp4...

🔍 Detecting shot boundaries in Batman (1989)-002_part_379.mp4...
✅ Detected 0 sequences in Batman (1989)-002_part_379.mp4.

📂 Organizing frames for Batman (1989)-002_part_379.mp4...



📂 Organizing Sequences: 0it [00:00, ?it/s]



✅ JSON metadata saved at: /Users/mario/Desktop/Desktop/UofA/4.Winter-2025/ece-910/Batman/histogram-output-sequences/Batman (1989)-002_part_379.mp4/Batman (1989)-002_part_379.mp4.json


🔑 Processing Sequences (Batman (1989)-002_part_379.mp4): 100%|██████████| 1/1 [00:00<00:00, 5584.96it/s]
🔑 Processing Sequences (Batman (1989)-002_part_379.mp4): 100%|██████████| 1/1 [00:00<00:00, 16980.99it/s]
🔑 Processing Sequences (Batman (1989)-002_part_379.mp4): 100%|██████████| 1/1 [00:00<00:00, 7639.90it/s]
🔑 Processing Sequences (Batman (1989)-002_part_379.mp4): 100%|██████████| 1/1 [00:00<00:00, 11491.24it/s]
🔑 Processing Sequences (Batman (1989)-002_part_379.mp4): 100%|██████████| 1/1 [00:00<00:00, 6754.11it/s]
🔑 Processing Sequences (Batman (1989)-002_part_379.mp4): 100%|██████████| 1/1 [00:00<00:00, 9258.95it/s]
🔑 Processing Sequences (Batman (1989)-002_part_379.mp4): 100%|██████████| 1/1 [00:00<00:00, 11244.78it/s]
🔑 Processing Sequences (Batman (1989)-002_part_379.mp4): 100%|██████████| 1/1 [00:00<00:00, 12372.58it/s]
🔑 Processing Sequences (Batman (1989)-002_part_379.mp4): 100%|██████████| 1/1 [00:00<00:00, 9776.93it/s]
🔑 Processing Sequences (Batman (1989)-002_part_379.


✅ Process complete!
